In [ ]:
import tensorflow as tf
import numpy as np
import os
import time

In [ ]:
#from google.colab import files
#files.upload()

In [ ]:
text = open('shakespeare_input.txt', 'rb').read().decode(encoding='utf-8')
print('Total unique charachter: ',len(set(text)))

Total unique charachter:  67


In [ ]:
vocab = sorted(set(text))
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [ ]:
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)


In [ ]:
seq_length = 100
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [ ]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

Creating Data set for training

In [ ]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [ ]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

GRU

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)
  
model.summary()


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (64, None, 256)           17152     
_________________________________________________________________
gru_4 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_6 (Dense)              (64, None, 67)            68675     
Total params: 4,024,131
Trainable params: 4,024,131
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir_gru = './training_checkpoints_gru'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir_gru, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
history = model.fit(dataset, epochs=10, callbacks=[checkpoint_callback])

Epoch 1/10
707/707 [==============================] - 41s 58ms/step - loss: 1.9910
Epoch 2/10
707/707 [==============================] - 43s 60ms/step - loss: 1.4603
Epoch 3/10
707/707 [==============================] - 42s 59ms/step - loss: 1.3628
Epoch 4/10
707/707 [==============================] - 42s 60ms/step - loss: 1.3169
Epoch 5/10
707/707 [==============================] - 42s 60ms/step - loss: 1.2859
Epoch 6/10
707/707 [==============================] - 42s 60ms/step - loss: 1.2617
Epoch 7/10
707/707 [==============================] - 42s 60ms/step - loss: 1.2419
Epoch 8/10
707/707 [==============================] - 42s 60ms/step - loss: 1.2245
Epoch 9/10
707/707 [==============================] - 42s 60ms/step - loss: 1.2097
Epoch 10/10
707/707 [==============================] - 42s 60ms/step - loss: 1.1963


LSTM

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)
  
model.summary() 

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (64, None, 256)           17152     
_________________________________________________________________
lstm_4 (LSTM)                (64, None, 1024)          5246976   
_________________________________________________________________
dense_10 (Dense)             (64, None, 67)            68675     
Total params: 5,332,803
Trainable params: 5,332,803
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir_lstm = './training_checkpoints_lstm'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir_lstm, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
history = model.fit(dataset, epochs=10, callbacks=[checkpoint_callback])

Epoch 1/10
707/707 [==============================] - 52s 74ms/step - loss: 1.9859
Epoch 2/10
707/707 [==============================] - 52s 74ms/step - loss: 1.4526
Epoch 3/10
707/707 [==============================] - 53s 75ms/step - loss: 1.3484
Epoch 4/10
707/707 [==============================] - 53s 75ms/step - loss: 1.2979
Epoch 5/10
707/707 [==============================] - 53s 75ms/step - loss: 1.2639
Epoch 6/10
707/707 [==============================] - 53s 75ms/step - loss: 1.2371
Epoch 7/10
707/707 [==============================] - 53s 75ms/step - loss: 1.2140
Epoch 8/10
707/707 [==============================] - 53s 75ms/step - loss: 1.1929
Epoch 9/10
707/707 [==============================] - 53s 75ms/step - loss: 1.1733
Epoch 10/10
707/707 [==============================] - 53s 75ms/step - loss: 1.1540


Bidirectional_LSTM

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform')),
                               
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)
  
model.summary() 

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (64, None, 256)           17152     
_________________________________________________________________
bidirectional (Bidirectional (64, None, 2048)          10493952  
_________________________________________________________________
dense_8 (Dense)              (64, None, 67)            137283    
Total params: 10,648,387
Trainable params: 10,648,387
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir_bd_lstm = './training_checkpoints_bd_lstm'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir_bd_lstm, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
history = model.fit(dataset, epochs=10, callbacks=[checkpoint_callback])

Epoch 1/10
707/707 [==============================] - 102s 144ms/step - loss: 0.2955
Epoch 2/10
707/707 [==============================] - 102s 144ms/step - loss: 0.0515
Epoch 3/10
707/707 [==============================] - 102s 144ms/step - loss: 0.0450
Epoch 4/10
707/707 [==============================] - 102s 145ms/step - loss: 0.0411
Epoch 5/10
707/707 [==============================] - 103s 145ms/step - loss: 0.0384
Epoch 6/10
707/707 [==============================] - 102s 145ms/step - loss: 0.0361
Epoch 7/10
707/707 [==============================] - 102s 145ms/step - loss: 0.0339
Epoch 8/10
707/707 [==============================] - 102s 145ms/step - loss: 0.0320
Epoch 9/10
707/707 [==============================] - 102s 145ms/step - loss: 0.0302
Epoch 10/10
707/707 [==============================] - 102s 145ms/step - loss: 0.0282


Text generation Function

In [ ]:
def generate_text(model, start_string):
  
  num_generate = 1000

  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  text_generated = []

  
  temperature = 1.0

  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)

    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

GRU TEXT

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir_gru)


'./training_checkpoints_gru/ckpt_10'

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir_gru))

model.build(tf.TensorShape([1, None]))

print(generate_text(model, start_string=u"R"))

ROKE:
How fair noteowledge in one shere,
That thy face
I stand on mere his shame, obedient child;
Who, this is the story, show me of this eagle
in thy mother's penal death.

DROMIO OF OLYO:
When must my househfend give troce.

VESTOR:
And to thy bones this charge about within a man shall find
Him sup against your majesty, he came
Richard by the season on: yet, to feed up
Is three monks you have done any deep die.

FABIAN:
I am Flanderer.

SIR ANDREW:
If you must encounter it a verk so dole!

PISTOL:
Is'tiniar?

GLOUCESTER:
Yes, my sweet Prince Paris, which has deliver'd ye and not dework in't. By yond a
marriage-voice to the gentle crab-treads of your thoughts:
Your grace is meeting, and at merry jests.

FALSTAFF:
Go certhinks to mine honour,
My true mothers have shut all; and not that diviner:
If by heaven, till you had attain'd.

LUCIANA:
Poins!
USALINE:
I will or sorrow up.

VALENTINE:
Path new time to twain,
And the ming can bring them,
When instruments the thibted scorrage:
In fai

LSTM TEXT

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir_lstm))

model.build(tf.TensorShape([1, None]))

print(generate_text(model, start_string=u"R"))

RUGENTER:
I go to Juliet and be about thy entertainment.

VIOLA:
How has this? what names fair?

AUTOLY:
I am old, I should not thither stoop.

First Playe:
That he and
I have been O for mine enemies, come to him, while none doth fright
The insolence of Madam Silvia
The greater fates of Maines, bus man, how all they want
in lackey! I have done; I sweare a hundred colours
With half dishonour to the groves, ithe swords,
And like an imprescrieunce, when he is ne'er out of
kisched with the cruelty; otherwise and counsel or stiff for heart and
beard and appelet sold to picture of him: I say hanging himself.

WESTMORELAND:
Hed all the swelter creeping wings!

PETERTI:
We two will quickly find you.

KING HENRY VI:
Put it you, good Knight.

Servant:
By all is as many may help me:
I have been sword on me; I should endure

LAERTES:

Third Secretled you are delivered,
By all haste: well.

SEBASTIAN:
Doubt not yea, that he and my suit to thee,
I shull fall England in which special eyes,
A most dis

BIDIRECTIONAL LSTM

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir_bd_lstm))

model.build(tf.TensorShape([1, None]))

print(generate_text(model, start_string=u"R"))

RORORORORORORORORORORORORORORORORORORORORORORORORORORORORORORONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONONENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENENE